In [1]:
import torch
import torch.nn as nn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

from einops import rearrange, repeat, reduce

from darts.datasets import AirPassengersDataset

In [3]:
a = torch.randn(5,5)
a

tensor([[ 0.2735, -0.5114, -0.1660,  1.3099, -0.6959],
        [-0.7905, -0.3967,  1.7651,  0.5602, -0.2314],
        [-0.4153,  0.3058, -0.3118, -1.0929, -0.0879],
        [ 0.3062, -0.8814,  1.2134,  1.4401,  0.4152],
        [ 0.3847,  0.1428, -0.0129, -0.8028, -0.2110]])

In [4]:
b = torch.randn(5,5)
b

tensor([[-1.3463, -0.1377,  0.2445,  0.8879,  1.4370],
        [-1.3745,  0.6306,  0.8739,  0.2711, -0.5804],
        [-0.6964, -1.7672,  1.8254,  1.4703, -0.6612],
        [ 0.1549, -0.8364,  0.9529, -2.5726, -0.8154],
        [ 1.1495, -1.0742, -0.1568,  0.1535,  1.2663]])

In [6]:
torch.linalg.matrix_norm(a)

tensor(3.7204)

In [7]:
torch.linalg.matrix_norm(b)

tensor(5.5484)